# Using the Python API

<a target="_blank" href="https://colab.research.google.com/github/RobbinBouwmeester/Workshop_Grenoble_2025/blob/main/in-depth-python-api.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

In [ ]:
!pip install plotly ms2rescore==3.1.4 im2deep==1.0.2 psm_utils==1.4.0


In [ ]:
import logging
import plotly.io
import os
import requests
import tarfile
import zipfile


logging.basicConfig(level=logging.INFO)
#plotly.io.renderers.default = "plotly_mimetype+notebook"
plotly.io.renderers.default = "colab"

## Reading and parsing peptide-spectrum matches

In [ ]:
from psm_utils.io import read_file

from ms2rescore.report.charts import score_histogram

In [ ]:
import urllib.request

url = "https://github.com/compomics/Workshop_Grenoble_2025/raw/refs/heads/main/data/Velos005137.mgf"
output_path = "Velos005137.mgf"

urllib.request.urlretrieve(url, output_path)
print(f"Downloaded to: {output_path}")

In [ ]:
url = "https://github.com/compomics/Workshop_Grenoble_2025/raw/refs/heads/main/database/combined_fasta.zip"
output_path = "combined_fasta.zip"
extract_dir = "./"


urllib.request.urlretrieve(url, output_path)
print(f"Downloaded to: {output_path}")

# Open and extract the contents
with zipfile.ZipFile(output_path, 'r') as zip_ref:
    zip_ref.extractall(extract_dir)

print(f"Extraction completed successfully to '{extract_dir}'.")

In [ ]:
# Define URL and file paths
url = "https://github.com/lazear/sage/releases/download/v0.14.7/sage-v0.14.7-x86_64-unknown-linux-gnu.tar.gz"
archive_path = "sage-v0.14.7.tar.gz"
extract_dir = "./"
bin_sage_loc = "sage-v0.14.7-x86_64-unknown-linux-gnu/sage"

# Step 1: Download the file
print("Downloading archive...")
with requests.get(url, stream=True) as r:
    r.raise_for_status()
    with open(archive_path, 'wb') as f:
        for chunk in r.iter_content(chunk_size=8192):
            f.write(chunk)

print(f"Downloaded archive to: {archive_path}")

# Step 2: Extract the tar.gz file
print("Extracting archive...")
with tarfile.open(archive_path, "r:gz") as tar:
    tar.extractall(path=extract_dir)

print(f"Extracted to directory: {extract_dir}")

# Set executable permission (equivalent to chmod +x)
os.chmod(bin_sage_loc, os.stat(bin_sage_loc).st_mode | 0o111)

In [ ]:
# Define URL and file paths
url = "https://github.com/lazear/sage/releases/download/v0.14.7/sage-v0.14.7-x86_64-pc-windows-msvc.zip"
archive_path = "sage-v0.14.7.zip"
extract_dir = "./"
bin_sage_loc = "sage-v0.14.7-x86_64-pc-windows-msvc\\sage.exe"

# Step 1: Download the ZIP file
print("Downloading archive...")
with requests.get(url, stream=True) as r:
    r.raise_for_status()
    with open(archive_path, 'wb') as f:
        for chunk in r.iter_content(chunk_size=8192):
            f.write(chunk)

print(f"Downloaded archive to: {archive_path}")

# Step 2: Extract the ZIP file
print("Extracting archive...")
with zipfile.ZipFile(archive_path, "r") as zip_ref:
    zip_ref.extractall(path=extract_dir)

print(f"Extracted to directory: {extract_dir}")

In [ ]:
url = "https://github.com/compomics/Workshop_Grenoble_2025/raw/refs/heads/main/json/params.json"
output_path = "params.json"

urllib.request.urlretrieve(url, output_path)
print(f"Downloaded to: {output_path}")

In [ ]:
!{bin_sage_loc} params.json

### Reading the PSM file

MS²Rescore is fully centered around the use of a `psm_utils` PSMList. This is a unified data representation of PSMs and their various attributes. Internally, it is simply a list of Pydantic data classes which represent PSMs. With the submodule `psm_utils.io`, we can read PSMs from a variety of file formats. Here, we will read a PSM file in the MaxQuant `msms.txt` format.

Importantly, for rescoring, the PSM file must contain all target and decoy PSMs, including PSMs that did not pass the FDR threshold. Most search engines must be specifically configured to return all PSMs without FDR filtering.


In [ ]:
psm_list = read_file("results.sage.tsv", filetype="sage_tsv")
psm_list["spectrum_id"] = [str(spec_id) for spec_id in psm_list["spectrum_id"]]

For a quick inspection, we can format the PSM list as a Pandas dataframe and display the first few rows:

In [ ]:
psm_list.to_dataframe().head()

We can also directly plot the current PSM score distributions:

In [ ]:
score_histogram(psm_list)

### Parsing modification names

While `psm_utils` could take care of all file parsing, we must still map the amino acid modification names that were used by the search engine to ones that are recognized by tools such as MS²PiP and DeepLC. This includes:

- Names as used in the Unimod or PSI-MOD databases
- Accession numbers as used in the Unimod or PSI-MOD databases
- Chemical formulas
- Mass shifts in Da

Note that, for instance DeepLC, requires a chemical formula to encode modifications. It will not be able to correctly encode modifications if only the mass shift is provided. It is therefore always preferred to provide a name/accession of a database where this information can be retrieved, or provide the chemical formula directly.

If a chemical formula is provided, other tools, such as MS²PIP, can use it to derive the correct mass shift.

To map modification names, simply provide a dictionary to the `psm_list.map_modifications` method.

In [ ]:
psm_list.rename_modifications({
    "-17.026548": "Gln->pyro-Glu",
    "-18.010565": "Glu->pyro-Glu",
    "15.9949": "Oxidation",
    "57.0215": "Carbamidomethyl",
    "de": "Deamidation",
})

### Assigning fixed modifications

Some search engines (although not many) do not report fixed (sometimes also called static) modifications in their output PSM files. This is for instance the case for MaxQuant. With the method `psm_list.add_fixed_modifications`, we can systematically assign fixed modifications to their amino acid targets, as was done during the search.

Note that `add_fixed_modifications` adds the modification in the ProForma 2.0 encoding as a prefix. To fully apply the modifications in the sequence, use the `psm_list.apply_fixed_modifications` method. A PSM `ACDE/2` would therefore go from `<[U:4]@C>ACDE/2` to `AC[U:4]DE/2`.



In [ ]:
psm_list.add_fixed_modifications([("U:Carbamidomethyl", ["C"])])
psm_list.apply_fixed_modifications()

## Adding rescoring features

In [ ]:
import pandas as pd

from ms2rescore.feature_generators.basic import BasicFeatureGenerator
from ms2rescore.feature_generators.maxquant import MaxQuantFeatureGenerator
from ms2rescore.feature_generators.ms2pip import MS2PIPFeatureGenerator
from ms2rescore.feature_generators.deeplc import DeepLCFeatureGenerator

The PSM list, as read from a MaxQuant msms.txt file, currently does not contain any rescoring features:

In [ ]:
pd.DataFrame(list(psm_list["rescoring_features"]))

Note that `psm_list[rescoring_features]` returns a Numpy array of dictionaries, but we can use Pandas to format it as a table.

### Basic features

Basic features from `BasicFeatureGenerator` can be added from any PSM list and contain simple feature, such as the search engine score, charge state, and absolute precursor mass error.

In [ ]:
basic_fgen = BasicFeatureGenerator()
basic_fgen.add_features(psm_list)

In [ ]:
pd.DataFrame(list(psm_list["rescoring_features"]))

Now the PSMs contain a simple set of features.

Note that the charge state is present as both an integer and a one-hot encoded vector. This is because charge state can act as both a categorical and a numerical feature. Simply put, sometimes an effect increases or decreases with increasing charge state, and sometimes it is simply different for different charge states.

### MS²PIP features

MS²PIP is a machine learning tool that predicts the MS2 spectrum of a peptide given its sequence. It is previously identified MS2 spectra and their corresponding peptide sequences. Because MS²PIP uses the highly performant - but traditional - machine learning approach XGBoost, it can already produce accurate predictions even if trained on smaller spectral libraries. This makes MS²PIP a very flexible platform to train new models on custom datasets. Nevertheless, MS²PIP comes with several pre-trained models, which we will use in this tutorial.

Because traditional proteomics search engines do not fully consider MS2 peak intensities in their scoring functions, adding rescoring features derived from spectrum prediction tools has proved to be a very effective way to further improve the sensitivity of peptide-spectrum matching. Generating features from MS²PIP predictions follows these steps:

1. Predict MS2 spectra for all PSMs in the dataset, including decoy PSMs and low scoring target PSMs.
2. Read and parse the observed MS2 spectra from the original spectrum files (MGF or mzML).
3. Compare the predicted and observed spectra using various similarity metrics, which are returned as rescoring features.


#### Configuring MS²PIP

In contrast to the basic feature generator, MS²PIP requires some parameters to be set and requires access to the original observed peptide spectra:

- `model`: Name of the prediction model to be used. This strongly depends on the dataset your are rescoring. A list of all MS²PIP models is available on https://ms2pip.readthedocs.io/en/latest/prediction-models/.
- `ms2_tolerance`: As MS²PIP must reannotate the observed MS2 spectra, a mass tolerance must be set. For MS²PIP, this is configured in Dalton. A good value for Orbitrap spectra, for example, is `0.02`.
- `spectrum_path`: Path to the original spectrum files. This can be a single file, or a directory containing multiple files. The spectrum files must be in MGF or mzML format.
- `spectrum_id_pattern`: A regular expression pattern to extract the spectrum ID from the PSMs (see below).
- `processes`: Number of CPU processes to use for parallel processing. Note that higher values can lead to memory issues.

In [ ]:
ms2pip_fgen = MS2PIPFeatureGenerator(
    model="HCD",
    ms2_tolerance=0.02,
    spectrum_path="Velos005137.mgf",
#    spectrum_id_pattern=r".*",
#    psm_id_pattern=r".*",
    processes=8,
)

A very import aspect of generating MS²PIP-derived features is making sure the peptide identifications are matched to the correct spectra. Unfortunately, many search engines alter spectrum identifiers in their output files. In the case of MaxQuant and data from Thermo instruments, fortunately, the scan numbers are returned. If the spectrum files have been converted to MGF with ThermoRawFileParser, we need to extract these exact scan numbers from the spectrum `TITLE` entries. This can be done with a regular expression pattern. For example, the following pattern extracts the scan number from the following spectrum title:

```python
spectrum_id_pattern = r"scan=(\d+)"
```

```
mzspec=20161213_NGHF_DBJ_SA_Exp3A_HeLa_1ug_7min_15000_02.raw: controllerType=0 controllerNumber=1 scan=2
```

We can test this with the first spectrum in the file:

In [ ]:
from pyteomics.mgf import read as read_mgf

for spectrum in read_mgf("Velos005137.mgf"):
    spectrum_id = spectrum["params"]["title"]
    break
print(spectrum_id)

We can use this scan number to retrieve its corresponding PSM:

In [ ]:
psm_list[psm_list["spectrum_id"] == "controllerType=0 controllerNumber=1 scan=2162"]

In [ ]:
psm_list["spectrum_id"]

#### Generating MS²PIP features

In [ ]:
ms2pip_fgen.add_features(psm_list)

In [ ]:
pd.DataFrame(list(psm_list["rescoring_features"]))

### Generating DeepLC features

In [ ]:
deeplc_fgen = DeepLCFeatureGenerator(
    lower_score_is_better=False,
    spectrum_path=None,
    processes=8,
    deeplc_retrain=False,
)

deeplc_fgen.add_features(psm_list)

### Assessing individual feature performance before rescoring

In [ ]:
import plotly.express as px
from ms2rescore.report.charts import (
    calculate_feature_qvalues,
    feature_ecdf_auc_bar,
    fdr_plot,
    ms2pip_correlation,
)
import deeplc.plot

features = pd.DataFrame(list(psm_list["rescoring_features"]))

The following plot shows the number of identified PSM at varying FDR thresholds, where the red line indicates the commonly used 1% FDR threshold.

In [ ]:
psm_list.calculate_qvalues()  # msms.txt does not contain q-values
fig = fdr_plot(psm_list.to_dataframe(), fdr_thresholds=None, log=False)
fig.show()

We can make a similar plot for each of the rescoring features by simply calculating q-values as if the feature were a search engine score.

In [ ]:
feature_qvalues, feature_ecdf_auc = calculate_feature_qvalues(
    features=features,
    is_decoy=psm_list["is_decoy"],
)

In [ ]:
feature_names = {
    "basic": basic_fgen.feature_names,
    "ms2pip": ms2pip_fgen.feature_names,
    "deeplc": deeplc_fgen.feature_names,
}
feature_generator_map = {name: gen for gen, f_list in feature_names.items() for name in f_list}

feature_qvalues_melt = feature_qvalues.melt(var_name="feature", value_name="q-value")
feature_qvalues_melt["feature_generator"] = feature_qvalues_melt["feature"].map(feature_generator_map)

The larger the area under the curve (AUC) is for each line, the better. We can therefore visualize the same information in a more easy-to-interpret bar chart with the AUCs, which were already calculated by the `calculate_feature_qvalues` function.

In [ ]:
feature_ecdf_auc["feature_generator"] = feature_ecdf_auc["feature"].map(feature_generator_map)

In [ ]:
fig = feature_ecdf_auc_bar(feature_ecdf_auc)
fig.show()

Importantly, a low q-value ECDF AUC does not necessarily mean that a feature is not valuable for rescoring. The more orthogonal a feature is to the other information already used in PSM scoring, the more it will help to confidently identify PSMs. For instance, retention time features are completely independent from the information used in a traditional search engine. When combined with other scoring information, they can provide a substantial boost in sensitivity.

The quality of specific rescoring features can also be assessed in a manner specific to the feature. For instance, we can plot the distribution of the MS²PIP pearson correlations for target PSMs that initially passed the FDR threshold:

In [ ]:
fig = ms2pip_correlation(
    features=features,
    is_decoy=psm_list["is_decoy"],
    qvalue=psm_list["qvalue"]
)
fig.show()

For DeepLC, we can plot the predicted retention time versus the observed retention time:

In [ ]:
sum(psm_list["is_decoy"])

In [ ]:
fig = deeplc.plot.scatter(
    df=features[(~psm_list["is_decoy"]) & (psm_list["qvalue"] <= 0.01)],  # noqa: E712
    predicted_column="predicted_retention_time_best",
    observed_column="observed_retention_time_best",
)
fig.show()

DeepLC also provides a function to plot the current relative mean absolute error (rMAE) of the predicted retention times against a distribution of 460 benchmark datasets. The lower the rMAE, the better.

In [ ]:
fig = deeplc.plot.distribution_baseline(
    df=features[(~psm_list["is_decoy"]) & (psm_list["qvalue"] <= 0.01)],  # noqa: E712
    predicted_column="predicted_retention_time_best",
    observed_column="observed_retention_time_best",
)
fig.show()

## Rescoring

### Using Mokapot directly

In [ ]:
import matplotlib.pyplot as plt
from mokapot import brew
from ms2rescore.rescoring_engines.mokapot import convert_psm_list

%matplotlib inline

In this tutorial, we will use Mokapot as rescoring engine. By default it employs the same methodologies as Percolator, but is fully implemented in Python. First, the PSM list must be converted to a Mokapot `LinearPsmDataset`.

In [ ]:
linear_psm_dataset = convert_psm_list(psm_list)

Then, the Mokapot `brew` function can be called to rescore the PSMs:

In [ ]:
confidence_results, models = brew(linear_psm_dataset)

Mokapot contains a function to conveniently plot the results:

In [ ]:
confidence_results.plot_qvalues()
plt.show()

Or to compare the un-rescored PSMs to the rescored PSMs:

In [ ]:
linear_psm_dataset.assign_confidence().plot_qvalues(label="Before rescoring")
confidence_results.plot_qvalues(label="After rescoring")
plt.legend()
plt.show()

Using mokapot directly allows us to quickly generate `LinearPsmDataset` objects with different feature sets, for instance:

- Only basic features
- Basic + MaxQuant-derived features (mimicking a traditional Percolator rescoring run)
- Basic + MaxQuant-derived + MS²PIP features
- Basic + MaxQuant-derived + MS²PIP + DeepLC features


In [ ]:
feature_sets = {
    "basic": feature_names["basic"],
    "basic_ms2pip": feature_names["basic"] + feature_names["ms2pip"],
    "basic_ms2pip_deeplc": feature_names["basic"] + feature_names["ms2pip"] + feature_names["deeplc"],
}

In [ ]:
linear_psm_dataset.assign_confidence().plot_qvalues(label="Before rescoring")

for feature_set, features in feature_sets.items():
    linear_psm_dataset = convert_psm_list(psm_list, features)
    confidence_results, models = brew(linear_psm_dataset)
    confidence_results.plot_qvalues(label=f"Rescoring with: {feature_set}")

plt.xscale('log')
plt.xlim(1e-4, 1)
plt.legend()
plt.show()

### Using Mokapot within MS²Rescore

Mokapot can also be used for rescoring with the higher-level integration in MS²Rescore. Simply pass
a PSMList object with rescoring features added to the `rescore` function. Doing so will automatically
convert the PSMList to a Mokapot `LinearPsmDataset`, run the `brew` function, and update the original
PSMList with the new scores, q-values and PEPs.

In [ ]:
from ms2rescore.rescoring_engines import mokapot

mokapot.rescore(psm_list)

In [ ]:
score_histogram(psm_list.to_dataframe())